In [1]:
#  Create a pipeline that scraps all the masterpieces pages of the Artsper website, convert it into a dataframe 
#   and stock it in MySQL database"""

import requests
from bs4 import BeautifulSoup
import pandas as pd
import pymysql
from sqlalchemy import create_engine


def get_Art_Url(): 
    """Function getting the url of all MASTERPIECES in the website Artsper """
    
    total_listlinks =[]
    for k in range(74):
        url = (f'https://www.artsper.com/fr/oeuvres-d-art-contemporain?selection=masterpieces&sort=5&page={1+k}')
        html = requests.get(url).content
        soup = BeautifulSoup(html, 'lxml')
        total_listlinks.append(['https://www.artsper.com'+i.get('href') for i in soup.select('#catalog figure>a')])
    return sum(total_listlinks, [])


def get_Dtc_Total_Art(total_listlinks):
    """ Function scraping severals selected elements from masterpieces pages and stocking them into a dictionnary """
    
    json=[]
    for k in range(len(total_listlinks)):
        dct=dict()
        html = requests.get(total_listlinks[k]).content
        soup = BeautifulSoup(html, 'lxml')
        dct['Artist']=[i.text for i in soup.select('.primary-title')][0]
        dct['Name & Date'] = [i.text.strip('\n                    ') for i in soup.select('h1 .secondary-title')][0]
        dct['Category'] = [i.text for i in soup.select('a .category')][0]
        dct['Price'] = '0' if len(soup.select('#sticky p.media-price'))==0  else soup.select('#sticky p.media-price')[0].text
        dct['Theme'] = [i.text.strip('\n').replace('\n','') for i in soup.select('p.pull-right:nth-child(2)')][4]
        dct['Dimension'] = [i.text for i in soup.select('p.pull-right .measure:first-child')][0]
        dct['City Galery'] = [i.text.strip('\n ') for i in soup.select('.city')][0]
        json.append(dct)    
    return json 


def json_to_SQL(json):
    """Function creating a dataframe from the dictionnary and exporting it to MySQL database"""
    
    artsper = pd.DataFrame(json)
    username='Abelia'
    password='password'
    host='localhost'
    database_name='ARTSPER'
    engine=create_engine(f"""mysql+pymysql://{username}:{password}@{host}/{database_name}""")
    artsper.to_sql('data2', engine, index=False, if_exists='replace')
    
    
if __name__=='__main__':
    """Pipeline executing all the functions above"""
    
    data = get_Art_Url()
    json = get_Dtc_Total_Art(data)
    json_to_SQL(json)

KeyboardInterrupt: 